# Comparación de técnicas de imputación de valores perdidos

## Importaciones y carga del dataset

In [44]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import time
import numpy as np
#from sklearn.base import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [ ]:
RANDOM_STATE = 0

def simulate_missingness(X, missing_rate=0.05, seed=RANDOM_STATE):
    rng = np.random.RandomState(seed)
    X2 = X.astype(float).copy()
    n, d = X2.shape
    m = int(missing_rate * n * d)
    idx = rng.choice(n*d, m, replace=False)
    row_indices = idx // d
    col_indices = idx % d
    X2.values[row_indices, col_indices] = np.nan
    return X2

# 1. Load the "Bunch" object
data = load_breast_cancer()

df_features = pd.DataFrame(data.data, columns=data.feature_names); df_target = pd.Series(data.target, name='target')
df = pd.concat([df_features, df_target], axis=1)

X = df.drop('target', axis=1)
y = df['target']
print('X shape:', X.shape, '| y shape:', y.shape)

X shape: (569, 30) | y shape: (569,)

\-----------------------------Visualización de Valores perdidos-----------------------------/
No se encontraron valores perdidos. (Esto es inesperado)


## Base

In [46]:
X_base = X.copy()
y_base = y.copy()

In [47]:
X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(
    X_base, y_base, test_size=0.25, stratify=y_base, random_state=RANDOM_STATE
)

In [49]:
model = LogisticRegression(penalty='l2', solver='liblinear', random_state=RANDOM_STATE)
#Comienza la medición del tiempo
t0 = time.perf_counter()

model.fit(X_train_base, y_train_base);

#Finaliza la medición del tiempo
t_base = time.perf_counter() - t0

yp = model.predict(X_test_base)
acc_base = accuracy_score(y_test_base, yp)
f1_base = f1_score(y_test_base, yp, average='macro')

print('BASELINE')
print(f'Accuracy: {acc_base:.4f}  |  F1-macro: {f1_base:.4f}  |  tiempo: {t_base:.3f}s')

BASELINE
Accuracy: 0.9371  |  F1-macro: 0.9328  |  tiempo: 0.008s


In [ ]:
model = RandomForestClassifier()
#Comienza la medición del tiempo
t0 = time.perf_counter()

model.fit(X_train_base, y_train_base);

#Finaliza la medición del tiempo
t_base = time.perf_counter() - t0

yp = model.predict(X_test_base)
acc_base = accuracy_score(y_test_base, yp)
f1_base = f1_score(y_test_base, yp, average='macro')

print('BASELINE')
print(f'Accuracy: {acc_base:.4f}  |  F1-macro: {f1_base:.4f}  |  tiempo: {t_base:.3f}s')

BASELINE
Accuracy: 0.9510  |  F1-macro: 0.9473  |  tiempo: 0.085s


## Ahora simulamos que se pierden los valores 

In [48]:
X = simulate_missingness(X)
print("\n\\-----------------------------Visualización de Valores perdidos-----------------------------/")
has_missing = False
for columna in X.columns:
    perdidos = X[columna].isnull().sum() / len(X[columna]) * 100
    if perdidos != 0:
        print(f"La columna {columna} tiene {perdidos:.2f}% valores perdidos")
        has_missing = True

if not has_missing:
    print("No se encontraron valores perdidos. (Esto es inesperado)")


\-----------------------------Visualización de Valores perdidos-----------------------------/
La columna mean radius tiene 7.21% valores perdidos
La columna mean texture tiene 4.04% valores perdidos
La columna mean perimeter tiene 5.62% valores perdidos
La columna mean area tiene 5.27% valores perdidos
La columna mean smoothness tiene 5.62% valores perdidos
La columna mean compactness tiene 5.80% valores perdidos
La columna mean concavity tiene 4.92% valores perdidos
La columna mean concave points tiene 3.16% valores perdidos
La columna mean symmetry tiene 6.15% valores perdidos
La columna mean fractal dimension tiene 3.69% valores perdidos
La columna radius error tiene 4.57% valores perdidos
La columna texture error tiene 4.57% valores perdidos
La columna perimeter error tiene 5.62% valores perdidos
La columna area error tiene 6.33% valores perdidos
La columna smoothness error tiene 6.33% valores perdidos
La columna compactness error tiene 4.04% valores perdidos
La columna concavity 

## Imputación básica

In [50]:
X_basica = X.copy()
y_basica = y.copy()

In [51]:
X_train_basica, X_test_basica, y_train_basica, y_test_basica = train_test_split(
    X_basica, y_basica, test_size=0.25, stratify=y_base, random_state=RANDOM_STATE
)

### Eliminación de filas con NaN

In [52]:
X_train_basica1 = X_train_basica.dropna()
y_train_basica1 = y_train_basica.loc[X_train_basica1.index]

X_test_basica1 = X_test_basica.dropna()
y_test_basica1 = y_test_basica.loc[X_test_basica1.index]
print('X shape:', X_train_basica1.shape, '| y shape:', y_train_basica1.shape)
print('X shape:', X_test_basica1.shape, '| y shape:', y_test_basica1.shape)

X shape: (99, 30) | y shape: (99,)
X shape: (24, 30) | y shape: (24,)


In [53]:
model = LogisticRegression(penalty='l2', solver='liblinear', random_state=RANDOM_STATE)
#Comienza la medición del tiempo
t0 = time.perf_counter()

model.fit(X_train_basica1, y_train_basica1);

#Finaliza la medición del tiempo
t_basica1 = time.perf_counter() - t0

yp = model.predict(X_test_basica1)
acc_basica1 = accuracy_score(y_test_basica1, yp)
f1_basica1 = f1_score(y_test_basica1, yp, average='macro')

print('BASELINE')
print(f'Accuracy: {acc_basica1:.4f}  |  F1-macro: {f1_basica1:.4f}  |  tiempo: {t_basica1:.3f}s')

BASELINE
Accuracy: 0.9167  |  F1-macro: 0.9143  |  tiempo: 0.004s
